In [1]:
import json

# Path to your JSON file
file_path = 'H_chain_data/H4/ham/1.json'

# Read the file
with open(file_path, 'r') as f:
    data = json.load(f)

# Now `data` is a Python dict (or list, depending on the JSON content)
print(data)

{'HF': [5.732753118300001, [[-3.170758816108079, 2.4063667725116086e-12, -0.2126698141473511, 8.386722848315028e-12], [2.4060527875624718e-12, -2.0717032679097285, 3.979004625786579e-13, -0.30536996085684076], [-0.2126698141473503, 3.97279431574216e-13, -1.0747221483191334, 3.1549027984263735e-12], [8.384033918506913e-12, -0.30536996085685164, 3.162059156181281e-12, -0.7951096141342989]], [[[[0.8245402519958797, -3.726227208307601e-13, 0.1006731610679103, -1.933956342152246e-12], [-3.725667006554913e-13, 0.620825722952997, -2.795956106224418e-13, 0.1294758242230403], [0.10067316106791076, -2.7922193925352506e-13, 0.34320502474772374, -8.83333671494683e-13], [-1.933208721455224e-12, 0.12947582422304504, -8.751491336379529e-13, 0.2951438198081803]], [[-3.723656544475683e-13, 0.1423912254802397, -1.9548923593192256e-13, 0.051119054056220185], [0.14239122548023977, -8.262767473756609e-13, -0.02714410587682001, -5.360244489775805e-13], [-1.954731583638483e-13, -0.027144105876820174, -4.2504

In [2]:
import openfermion
from openfermion.transforms import get_fermion_operator, jordan_wigner, bravyi_kitaev
import itertools
import numpy as np
import torch

def add_spin_2bd(int_2bd):
    dim = int_2bd.shape[0]
    res = np.zeros((2*dim,2*dim,2*dim,2*dim))
    for i1,i2,i3,i4 in itertools.product(range(dim), repeat=4):
        for s1,s2,s3,s4 in itertools.product(range(2), repeat=4):
            if s1 == s4 and s2 == s3:
                res[i1*2+s1,i2*2+s2,i3*2+s3,i4*2+s4] = int_2bd[i1,i2,i3,i4]
    return res

def add_spin_1bd(int_1bd):
    dim = int_1bd.shape[0]
    res = np.zeros((2*dim,2*dim))
    for i1,i2 in itertools.product(range(dim), repeat=2):
        for s1,s2 in itertools.product(range(2), repeat=2):
            if s1 == s2:
                res[i1*2+s1,i2*2+s2] = int_1bd[i1,i2]
    return res

# do jordan wigner transformation
def JW_trans(Ham_const,int_1bd,int_2bd):
    # add spinint_2bd
    # note: openfermion and pyscf has different rules on int_2bd
    int_2bd = int_2bd.transpose((0,3,2,1))
    intop = openfermion.InteractionOperator(Ham_const,add_spin_1bd(int_1bd),add_spin_2bd(int_2bd)/2)
    #print(intop)
    fer = get_fermion_operator(intop)
    #print(fer)
    new_jw_hamiltonian = jordan_wigner(fer);
    return new_jw_hamiltonian

def LambdaQ(Ham_const,int_1bd,int_2bd,backend=torch):
    """
    Calculate the LambdaQ value for a given Hamiltonian.
    
    Parameters:
    Ham_const : torch.Tensor
        E0
        The constant part of the Hamiltonian.
    int_1bd : torch.Tensor
        h_{pq}
        The one-body interaction terms of the Hamiltonian.
    int_2bd : torch.Tensor
        g_{pqrs}
        The two-body interaction terms of the Hamiltonian.
        
    Returns: float
        \lambda_C+\lambda_T+\lambda_V^{\prime}
        where
            \lambda_C=\left|E0+\sum_p^N h_{p p}+\frac{1}{2} \sum_{p r}^N g_{p p r r}-\frac{1}{4} \sum_{p r}^N g_{p r r p}\right|,
            \lambda_T=\sum_{p q}^N\left|h_{p q}+\sum_r^N g_{p q r r}-\frac{1}{2} \sum_r^N g_{p r r q}\right|,
            \lambda_V^{\prime}=\frac{1}{2} \sum_{p>r, s>q}^N\left|g_{p q r s}-g_{p s r q}\right|+\frac{1}{4} \sum_{p q r s}^N\left|g_{p q r s}\right| .
    """

    trace = backend.trace
    einsum = backend.einsum
    abs = backend.abs
    sum = backend.sum

    h = int_1bd
    g = int_2bd
    N = h.shape[0]

    # λ_C = | sum_p h_pp + 1/2 sum_{p,r} g_pprr - 1/4 sum_{p,r} g_prrp |
    diag_h = trace(h)
    g_pprr = einsum('pprr->', g)
    g_prrp = einsum('prrp->', g)  # permute and sum over r → g[p,r,r,p]
    lambda_C = abs(diag_h + 0.5 * g_pprr - 0.25 * g_prrp+Ham_const)

    # λ_T = sum_{p,q} | h_pq + sum_r g_pqrr - 1/2 sum_r g_prrq |
    sum_r_g_pqrr = einsum('pqrr->pq', g)
    sum_r_g_prrq = einsum('prrq->pq', g)  # permute and sum over r → g[p,r,*,q]
    T_term = h + sum_r_g_pqrr - 0.5 * sum_r_g_prrq
    lambda_T = sum(abs(T_term))

    # λ_V' = 1/2 sum_{p>r, s>q} |g_pqrs - g_psrq| + 1/4 sum_{pqrs} |g_pqrs|
    indices = backend.arange(N)
    p, q, r, s = backend.meshgrid(indices, indices, indices, indices, indexing='ij')
    mask = (p > r) & (s > q)
    diff = g[p, q, r, s] - g[p, s, r, q]
    lambda_V_prime_1 = 0.5 * sum(abs(diff[mask]))
    lambda_V_prime_2 = 0.25 * sum(abs(g))
    lambda_V_prime = lambda_V_prime_1 + lambda_V_prime_2
    return lambda_C + lambda_T + lambda_V_prime



for method in ['l_opt','E_opt','HF','sto-3G']:

    Ham_const = data[method][0]
    int_1bd = np.array(data[method][1])
    int_2bd = np.array(data[method][2])
    qubit_ham = JW_trans(Ham_const,int_1bd,int_2bd)
    l_JW = sum(abs(coeff) for coeff in qubit_ham.terms.values())
    lambdaQ = LambdaQ(Ham_const,int_1bd,int_2bd,backend=np)
    print(method,l_JW,lambdaQ)


l_opt 11.052999522348381 11.052999668613504
E_opt 11.25525248962005 11.255252489639524
HF 7.838791670557886 7.838791670682111
sto-3G 19.651493594825975 19.651493594826192


In [3]:
for method in ['l_opt','E_opt','HF','sto-3G']:

    Ham_const = data[method][0]
    int_1bd = torch.tensor(data[method][1])
    int_2bd = torch.tensor(data[method][2])
    lambdaQ = LambdaQ(Ham_const,int_1bd,int_2bd,backend=torch)
    print(method,lambdaQ)

l_opt tensor(11.0530)
E_opt tensor(11.2553)
HF tensor(7.8388)
sto-3G tensor(19.6515)
